In [12]:
def link_generator():
    price_lo = 0
    help = []
    for price_lo in range(1, 59000, 1000):
        for page in range(1, 51):
            help.append(f"https://olx.ba/pretraga?attr=&attr_encoded=1&category_id=18&page={page}&price_to={price_lo + 1000}&price_from={price_lo}")
        for page in range(1, 51):
             help.append(f"https://olx.ba/pretraga?attr=&attr_encoded=1&category_id=18&page={page}&price_from=59000")
    return help

link_helpers = link_generator()
print(link_helpers[:5])

['https://olx.ba/pretraga?attr=&attr_encoded=1&category_id=18&page=1&price_to=1001&price_from=1', 'https://olx.ba/pretraga?attr=&attr_encoded=1&category_id=18&page=2&price_to=1001&price_from=1', 'https://olx.ba/pretraga?attr=&attr_encoded=1&category_id=18&page=3&price_to=1001&price_from=1', 'https://olx.ba/pretraga?attr=&attr_encoded=1&category_id=18&page=4&price_to=1001&price_from=1', 'https://olx.ba/pretraga?attr=&attr_encoded=1&category_id=18&page=5&price_to=1001&price_from=1']


In [21]:
print(len(link_helpers))

5900


In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from tqdm import tqdm


firefox_options = webdriver.FirefoxOptions()
firefox_options.add_argument("--headless") 

driver = webdriver.Firefox(options=firefox_options)


batch_num = 2


links_file_path = 'links1.txt'

# Define headers for the CSV file


# Open the CSV file for writing

    # Iterate over each link and extract data
for link in tqdm(link_helpers[100*(batch_num - 1):100*batch_num], total=len(link_helpers[100*(batch_num - 1):100*batch_num])):
    link = link.strip()  # Remove any leading/trailing whitespace
    if not link:
        continue  # Skip empty lines

    try:
        # Open the URL
        driver.get(link)
        
        # Wait for the page to fully load
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        
        # Give some extra time for JavaScript to execute (adjust if necessary)
        time.sleep(2)

        a_tags = driver.find_elements(By.XPATH, "//a[contains(@href, '/artikal/')]")

        # Extract the href attribute and store in a list
        links = [a_tag.get_attribute('href') for a_tag in a_tags]
        with open(links_file_path, 'a') as file:
            for temp in links:
                file.write(temp + "\n")
        
    except Exception as e:
        print(f"Failed to retrieve {link}: {e}")

# Quit the WebDriver session
driver.quit()

print("Finished processing all links.")


 42%|████▏     | 83/200 [06:42<12:07,  6.22s/it]

In [24]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from tqdm import tqdm

# Set up the Firefox options for headless mode (optional)
firefox_options = webdriver.FirefoxOptions()
firefox_options.add_argument("--headless")  # Use headless mode for no GUI

# Initialize the WebDriver for Firefox using the geckodriver from PATH
driver = webdriver.Firefox(options=firefox_options)

# Define the input file path
input_file_path = links_file_path  # Ensure this file exists with URLs

# Define the output CSV file path
output_file_path = 'car_data.csv'

# Open the input file and read all lines
with open(input_file_path, 'r') as file:
    links = file.readlines()

# Define headers for the CSV file
headers = ["URL", "Price", "Transmission", "Year", "Motor Strength (KW)", "Mileage", "Engine Capacity", "Manufacturer", "Model", "Fuel Type"]

# Open the CSV file for writing
with open(output_file_path, 'a', newline='', encoding='utf-8') as csvfile:
    # Create a CSV writer object
    csvwriter = csv.writer(csvfile)
    
    # Write the headers to the CSV file
    csvwriter.writerow(headers)

    # Iterate over each link and extract data
    for link in tqdm(links, total=len(links)):
        link = link.strip()  # Remove any leading/trailing whitespace
        if not link:
            continue  # Skip empty lines

        try:
            # Open the URL
            driver.get(link)
            
            # Wait for the page to fully load
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            
            # Give some extra time for JavaScript to execute (adjust if necessary)
            time.sleep(2)

            # Extract the required information
            try:
                # Extract Price
                price_element = driver.find_element(By.CSS_SELECTOR, "span.price-heading.vat")
                price = price_element.text
            except:
                price = "N/A"
            
            try:
                # Extract Transmission
                transmission_element = driver.find_element(By.XPATH, "//td[text()='Transmisija']/following-sibling::td")
                transmission = transmission_element.text.strip()
            except:
                transmission = "N/A"
            
            try:
                # Extract Year
                year_element = driver.find_element(By.XPATH, "//td[text()='Godište']/following-sibling::td")
                year = year_element.text.strip()
            except:
                year = "N/A"
            
            try:
                # Extract Motor Strength
                motor_strength_element = driver.find_element(By.XPATH, "//td[text()='Snaga motora (KW)']/following-sibling::td")
                motor_strength = motor_strength_element.text.strip()
            except:
                motor_strength = "N/A"

            try:
                # Extract Mileage
                mileage_element = driver.find_element(By.XPATH, "//td[text()='Kilometraža']/following-sibling::td")
                mileage = mileage_element.text.strip()
            except:
                mileage = "N/A"

            try:
                # Extract Engine Capacity
                engine_capacity_element = driver.find_element(By.XPATH, "//td[text()='Kubikaža']/following-sibling::td")
                engine_capacity = engine_capacity_element.text.strip()
            except:
                engine_capacity = "N/A"

            try:
                # Extract Manufacturer
                manufacturer_element = driver.find_element(By.XPATH, "//td[text()='Proizvođač']/following-sibling::td/a")
                manufacturer = manufacturer_element.text.strip()
            except:
                manufacturer = "N/A"

            try:
                # Extract Model
                model_element = driver.find_element(By.XPATH, "//td[text()='Model']/following-sibling::td/a")
                model = model_element.text.strip()
            except:
                model = "N/A"

            try:
                # Extract Fuel Type
                fuel_type_element = driver.find_element(By.XPATH, "//td[text()='Gorivo']/following-sibling::td")
                fuel_type = fuel_type_element.text.strip()
            except:
                fuel_type = "N/A"

            if fuel_type == "N/A" or model == "N/A" or manufacturer == "N/A" or price == "N/A" or transmission == "N/A" or year == "N/A" or motor_strength == "N/A" or mileage == "N/A" or engine_capacity == "N/A":
                continue
            
            # Write the extracted information to the CSV file
            row = [link, price, transmission, year, motor_strength, mileage, engine_capacity, manufacturer, model, fuel_type]
            csvwriter.writerow(row)

        except Exception as e:
            print(f"Failed to retrieve {link}: {e}")

# Quit the WebDriver session
driver.quit()

print("Finished processing all links.")


 66%|██████▋   | 1409/2122 [1:16:21<19:12:03, 96.95s/it]

Failed to retrieve https://olx.ba/artikal/56702120: Message: Navigation timed out after 300000 ms
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
TimeoutError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:740:5
bail@chrome://remote/content/marionette/sync.sys.mjs:211:19



 98%|█████████▊| 2075/2122 [1:53:16<1:12:10, 92.14s/it] 

Failed to retrieve https://olx.ba/artikal/54415254: Message: Navigation timed out after 300000 ms
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
TimeoutError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:740:5
bail@chrome://remote/content/marionette/sync.sys.mjs:211:19



100%|██████████| 2122/2122 [1:55:32<00:00,  3.27s/it]  


Finished processing all links.
